# Battle of the Neighborhoods - Week 3

## Part 1 - This section will web scrape the toronto Postal Codes and Boroughs from the wikipedia page

The wikipedia page used to obtain the information is;
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

The data was then passed into a data frame and data cleaned to remove NaN rows and combine rows. The shape of final dataframe is shown


In [65]:
import pandas as pd
import numpy as np
import requests
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df = df[df.Borough != "Not assigned"]
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [66]:
optim = df.groupby('Postal Code', axis=0)['Neighbourhood'].apply(list)
optim.head(20)

Postal Code
M1B                                     [Malvern, Rouge]
M1C             [Rouge Hill, Port Union, Highland Creek]
M1E                  [Guildwood, Morningside, West Hill]
M1G                                             [Woburn]
M1H                                          [Cedarbrae]
M1J                                [Scarborough Village]
M1K        [Kennedy Park, Ionview, East Birchmount Park]
M1L                    [Golden Mile, Clairlea, Oakridge]
M1M    [Cliffside, Cliffcrest, Scarborough Village West]
M1N                        [Birch Cliff, Cliffside West]
M1P    [Dorset Park, Wexford Heights, Scarborough Tow...
M1R                                  [Wexford, Maryvale]
M1S                                          [Agincourt]
M1T            [Clarks Corners, Tam O'Shanter, Sullivan]
M1V    [Milliken, Agincourt North, Steeles East, L'Am...
M1W                      [Steeles West, L'Amoreaux West]
M1X                                        [Upper Rouge]
M2H                

In [67]:
for i in range(0, len(df)):
    N = df['Neighbourhood'].iloc[i]
    for i2 in range(0, len(optim)):
        if str(N) in optim[i2]:
            df['Neighbourhood'].iloc[i] = optim[i2]
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
5,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]"
8,M9A,Etobicoke,"[Islington Avenue, Humber Valley Village]"
9,M1B,Scarborough,"[Malvern, Rouge]"
11,M3B,North York,[Don Mills]
12,M4B,East York,"[Parkview Hill, Woodbine Gardens]"
13,M5B,Downtown Toronto,"[Garden District, Ryerson]"


In [68]:
df['Neighbourhood'] = df['Neighbourhood'].astype(str).str.strip("]").str.strip("[").str.lstrip("'").str.rstrip("'")
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"""Queen's Park, Ontario Provincial Government"""
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [69]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)
df.head(15)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"""Queen's Park, Ontario Provincial Government"""
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [70]:
df.shape

(103, 3)

# Part 2 - get the lat and long for the different Postal Codes

This section uses the csv file that has the geographical coordinates for each postal code and merges it into 1 final data frame.

In [71]:
import csv
url = 'https://cocl.us/Geospatial_data'
glob = pd.read_csv(url)
glob.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [73]:
NDF = pd.merge(df, glob, on="Postal Code")
NDF.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"""Queen's Park, Ontario Provincial Government""",43.662301,-79.389494


# Part 3 - Explore and Cluster neighborhoods in Toronto

### Take the coordinates of the Toronto City Centre


Toronto Coordinates
https://www.latlong.net/place/toronto-on-canada-27230.html#:~:text=The%20latitude%20of%20Toronto%2C%20ON,%C2%B0%2020'%2049.2540''%20W.
Toronto, ON, Canada Latitude and longitude coordinates are: 43.651070, -79.347015.

Us this to place a map using folium and chart each of the boroghs present

In [74]:
import folium
#Toronto Lat long from web (can change for other city)
Lat = 43.651070
Long = -79.347015
Borough_map = folium.Map(location=[Lat, Long], zoom_start=10)


for lat, lng, borough, neighbourhood in zip(NDF['Latitude'], NDF['Longitude'], NDF['Borough'], NDF['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=9, popup=label, color='blue', fill=True).add_to(Borough_map)

Borough_map

Explore now

In [75]:
#Define foursquare credentials - Taken from the lab on manhatten and applied to toronto

CLIENT_ID = 'JYF4SUTSIUBZHRWW2QJ2D0YO4VYW1W0FIOI512Y11T34L05I' # your Foursquare ID
CLIENT_SECRET = 'NFQO2ABF0M0PW0D514EVCSERJ3U0GZLVDY02YST44JSSPYY1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JYF4SUTSIUBZHRWW2QJ2D0YO4VYW1W0FIOI512Y11T34L05I
CLIENT_SECRET:NFQO2ABF0M0PW0D514EVCSERJ3U0GZLVDY02YST44JSSPYY1


Explore a neighbourhood

In [76]:
NDF.loc[0, 'Neighbourhood']

'Parkwoods'

In [77]:
neighborhood_latitude = NDF.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NDF.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NDF.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [78]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=JYF4SUTSIUBZHRWW2QJ2D0YO4VYW1W0FIOI512Y11T34L05I&client_secret=NFQO2ABF0M0PW0D514EVCSERJ3U0GZLVDY02YST44JSSPYY1&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [79]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb6aa82ebf7ed0526231e83'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [81]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [82]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\thoma\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [83]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## Explore Neighbourhoods in ...

In [84]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
tor_venues = getNearbyVenues(names=NDF['Neighbourhood'],
                                   latitudes=NDF['Latitude'],
                                   longitudes=NDF['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
"Queen's Park, Ontario Provincial Government"
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birch

In [86]:
print(tor_venues.shape)
tor_venues.head()

(2139, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [87]:
tor_venues.groupby('Neighborhood').count()

##This is where the problem is coming with the neighbourthood 

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"""Clarks Corners, Tam O'Shanter, Sullivan""",12,12,12,12,12,12
"""Milliken, Agincourt North, Steeles East, L'Amoreaux East""",3,3,3,3,3,3
"""Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East""",2,2,2,2,2,2
"""Queen's Park, Ontario Provincial Government""",33,33,33,33,33,33
"""Steeles West, L'Amoreaux West""",12,12,12,12,12,12
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4
Woodbine Heights,8,8,8,8,8,8


In [88]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 274 uniques categories.


In [89]:
##analyse each neighbourhood

In [90]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head(20)

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
tor_onehot.shape

(2139, 274)

In [92]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"""Clarks Corners, Tam O'Shanter, Sullivan""",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"""Milliken, Agincourt North, Steeles East, L'Am...",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"""Old Mill South, King's Mill Park, Sunnylea, H...",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""Queen's Park, Ontario Provincial Government""",0.030303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""Steeles West, L'Amoreaux West""",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,York Mills West,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
tor_grouped.shape

(96, 274)

In [94]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----"Clarks Corners, Tam O'Shanter, Sullivan"----
                 venue  freq
0          Pizza Place  0.17
1             Pharmacy  0.08
2         Noodle House  0.08
3  Fried Chicken Joint  0.08
4                 Bank  0.08


----"Milliken, Agincourt North, Steeles East, L'Amoreaux East"----
                             venue  freq
0                       Playground  0.33
1                           Bakery  0.33
2                             Park  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East"----
                        venue  freq
0  Construction & Landscaping   0.5
1              Baseball Field   0.5
2                 Yoga Studio   0.0
3   Middle Eastern Restaurant   0.0
4         Monument / Landmark   0.0


----"Queen's Park, Ontario Provincial Government"----
               venue  freq
0        Coffee

4  Furniture / Home Store  0.08


----Lawrence Park----
                venue  freq
0                Park  0.33
1         Swim School  0.33
2            Bus Line  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2           Shopping Mall  0.06
3  Furniture / Home Store  0.06
4                    Bank  0.06


----Little Portugal, Trinity----
                   venue  freq
0                    Bar  0.09
1            Coffee Shop  0.07
2  Vietnamese Restaurant  0.04
3            Men's Store  0.04
4       Asian Restaurant  0.04


----Malvern, Rouge----
                             venue  freq
0             Fast Food Restaurant   1.0
1                      Yoga Studio   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South W

In [95]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"""Clarks Corners, Tam O'Shanter, Sullivan""",Pizza Place,Pharmacy,Gas Station,Noodle House,Chinese Restaurant,Bank,Italian Restaurant,Fried Chicken Joint,Intersection,Thai Restaurant
1,"""Milliken, Agincourt North, Steeles East, L'Am...",Park,Playground,Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,"""Old Mill South, King's Mill Park, Sunnylea, H...",Baseball Field,Construction & Landscaping,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
3,"""Queen's Park, Ontario Provincial Government""",Coffee Shop,Yoga Studio,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant
4,"""Steeles West, L'Amoreaux West""",Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bank,Chinese Restaurant,Indian Restaurant,Breakfast Spot,Gym,Pizza Place


## Cluster Neighborhoods

In [97]:
from sklearn.cluster import KMeans

In [98]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 3, 0, 1, 0, 1, 0, 0, 0])

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = NDF

# merge toronto_grouped with tor_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how = 'right')

tor_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,M1T,Scarborough,"""Clarks Corners, Tam O'Shanter, Sullivan""",43.781638,-79.304302,1,Pizza Place,Pharmacy,Gas Station,Noodle House,Chinese Restaurant,Bank,Italian Restaurant,Fried Chicken Joint,Intersection,Thai Restaurant
85,M1V,Scarborough,"""Milliken, Agincourt North, Steeles East, L'Am...",43.815252,-79.284577,2,Park,Playground,Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
101,M8Y,Etobicoke,"""Old Mill South, King's Mill Park, Sunnylea, H...",43.636258,-79.498509,3,Baseball Field,Construction & Landscaping,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M7A,Downtown Toronto,"""Queen's Park, Ontario Provincial Government""",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant
90,M1W,Scarborough,"""Steeles West, L'Amoreaux West""",43.799525,-79.318389,1,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bank,Chinese Restaurant,Indian Restaurant,Breakfast Spot,Gym,Pizza Place


In [104]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [107]:
# create map
map_clusters = folium.Map(location=[lat, lng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighbourhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Examine clusters 1 through 5

In [108]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Yoga Studio,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant
78,Scarborough,0,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Clothing Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
28,North York,0,Coffee Shop,Bank,Frozen Yogurt Shop,Shopping Mall,Bridal Shop,Sandwich Place,Diner,Restaurant,Deli / Bodega,Supermarket
39,North York,0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
55,North York,0,Sandwich Place,Italian Restaurant,Coffee Shop,Grocery Store,Thai Restaurant,Comfort Food Restaurant,Juice Bar,Butcher,Restaurant,Café
20,Downtown Toronto,0,Coffee Shop,Farmers Market,Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Bakery,Cheese Shop,Shopping Mall,Sporting Goods Shop
58,Scarborough,0,College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
43,West Toronto,0,Café,Coffee Shop,Breakfast Spot,Nightclub,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
87,Downtown Toronto,0,Airport Lounge,Airport Service,Boutique,Airport Terminal,Rental Car Location,Coffee Shop,Plane,Boat or Ferry,Sculpture Garden,Harbor / Marina
76,Mississauga,0,Hotel,Coffee Shop,Intersection,Gym,Fried Chicken Joint,Middle Eastern Restaurant,Mediterranean Restaurant,Burrito Place,American Restaurant,Gas Station


In [109]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,Scarborough,1,Pizza Place,Pharmacy,Gas Station,Noodle House,Chinese Restaurant,Bank,Italian Restaurant,Fried Chicken Joint,Intersection,Thai Restaurant
90,Scarborough,1,Fast Food Restaurant,Pharmacy,Coffee Shop,Sandwich Place,Bank,Chinese Restaurant,Indian Restaurant,Breakfast Spot,Gym,Pizza Place
93,Etobicoke,1,Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Pub,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
100,East Toronto,1,Skate Park,Light Rail Station,Butcher,Recording Studio,Auto Workshop,Burrito Place,Garden,Garden Center,Farmers Market,Fast Food Restaurant
26,Scarborough,1,Caribbean Restaurant,Hakka Restaurant,Bakery,Athletics & Sports,Gas Station,Bank,Thai Restaurant,Fried Chicken Joint,Doner Restaurant,Distribution Center
51,Scarborough,1,Motel,American Restaurant,Women's Store,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
67,Central Toronto,1,Department Store,Hotel,Dance Studio,Sandwich Place,Dog Run,Food & Drink Shop,Breakfast Spot,Park,Gym / Fitness Center,Electronics Store
65,Scarborough,1,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
40,North York,1,Grocery Store,Park,Hotel,Business Service,Shopping Mall,Liquor Store,Food Truck,Baseball Field,Bank,Athletics & Sports
46,North York,1,Grocery Store,Park,Hotel,Business Service,Shopping Mall,Liquor Store,Food Truck,Baseball Field,Bank,Athletics & Sports


In [110]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
85,Scarborough,2,Park,Playground,Bakery,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
21,York,2,Park,Women's Store,Pool,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
35,East York,2,Park,Convenience Store,Intersection,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
61,Central Toronto,2,Park,Swim School,Bus Line,Women's Store,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
49,North York,2,Park,Bakery,Construction & Landscaping,Women's Store,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
0,North York,2,Park,Food & Drink Shop,Women's Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
91,Downtown Toronto,2,Park,Trail,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
64,York,2,Park,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
66,North York,2,Park,Convenience Store,Women's Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [111]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,Etobicoke,3,Baseball Field,Construction & Landscaping,Women's Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
57,North York,3,Baseball Field,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Fast Food Restaurant


In [112]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,4,Print Shop,Women's Store,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
